In [ ]:
rm(list = ls())
setwd("/home/creambbq/code/bikes/archivos")
library(tidyverse)
library(sf)
ecoBiciData <- load("cleanData.Rdata")
bairesMap <- st_read("http://cdn.buenosaires.gob.ar/datosabiertos/datasets/barrios/barrios.geojson",
                     stringsAsFactors = FALSE)
bairesMap <- bairesMap %>% mutate(km2 = area/1000000)
print(head(bairesMap))
